In [2]:
import os
from dask_gateway import Gateway
# gateway = Gateway()
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)

In [6]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [7]:
os.environ["X509_USER_PROXY"] = "/depot/cms/private/users/shar1172/x509up_u2239311"
os.environ["PYTHONPATH"] += ":/depot/cms/users/shar1172/copperheadV2/"
os.environ['XRD_REQUESTTIMEOUT'] = "2400"
cluster = gateway.new_cluster(
    # conda_env = "/depot/cms/kernels/coffea2024",
    # conda_env = "/depot/cms/kernels/root632",
    conda_env = "/depot/cms/kernels/coffea_latest",
    worker_cores = 2, #1
    worker_memory = 6, #6
    env = dict(os.environ),
)

# cluster.adapt(100) 


In [8]:
cluster.adapt(minimum=3, maximum=100)
cluster

In [9]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

ClusterReport<name=cms.243fb482b73146018e726148399c405a, status=RUNNING>


In [15]:
os.environ['PWD']

'/home/shar1172'

In [4]:
"""
To shutdown the cluster, run this cell
Equivalently, you can also do "squeue -u <username>" 
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [5]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [16]:
options = gateway.cluster_options()  

In [18]:
# options.worker_cores
print(options)

In [19]:
! squeue -u shar1172

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
